# YOLOv9 for Pothole Detection

This notebook implements the YOLOv9 object detection model for detecting potholes. 
It implements the **Proposed Architecture** where `ADown` layers are replaced with `Conv` layers (kernel=3, stride=2) as described in the paper.

## 1. Setup Environment

In [ ]:
# Clone YOLOv9 repository
import os
if not os.path.exists('yolov9'):
    !git clone https://github.com/WongKinYiu/yolov9.git

%cd yolov9
!pip install -r requirements.txt
!pip install roboflow

## 2. Download Weights
We will use the pre-trained weights to speed up convergence.

In [ ]:
import torch

# Download weights if they don't exist
def download_weights(url, filename):
    if not os.path.exists(filename):
        try:
            torch.hub.download_url_to_file(url, filename)
            print(f"Downloaded {filename}")
        except Exception as e:
            print(f"Error downloading {filename}: {e}")
    else:
        print(f"{filename} already exists.")

download_weights('https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt', 'yolov9-c.pt')
download_weights('https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt', 'gelan-c.pt')

## 3. Prepare Dataset
You need a dataset annotated in YOLO format (txt files). 
You can upload your dataset to the `data/` folder or download one from Roboflow.

**Option A: Using Roboflow (Recommended)**
Replace `YOUR_API_KEY` below with your key from a public Pothole dataset project.

In [ ]:
from roboflow import Roboflow

# UNCOMMENT and Add your API KEY
# rf = Roboflow(api_key="YOUR_API_KEY")
# project = rf.workspace("workspace-name").project("project-name")
# dataset = project.version(1).download("yolov9")

## 4. Configure Model for Potholes
We create a custom data YAML file for the pothole class.

In [ ]:
import yaml

# Create dataset.yaml
data_config = {
    'train': '../train/images',  # adjust paths based on your dataset structure
    'val': '../valid/images',
    'test': '../test/images',
    'nc': 1,
    'names': ['pothole']
}

with open('data/pothole.yaml', 'w') as f:
    yaml.dump(data_config, f)

## 5. Implement Proposed Architecture & Train
We programmatically modify the `yolov9-c.yaml` model config significantly:
1. Read `models/detect/yolov9-c.yaml`
2. Replace all `ADown` layers with `Conv` layers (kernel=3, stride=2).
3. Save as `models/detect/yolov9-proposed.yaml`.
4. Train using this new architecture.

In [ ]:
import yaml

# 1. Load original config
# Note: The file path is relative to the current working directory which is 'yolov9/'
with open('models/detect/yolov9-c.yaml', 'r') as f:
    yolo_yaml = yaml.safe_load(f)

# 2. Function to replace ADown with Conv
def replace_adown(layers):
    new_layers = []
    for layer in layers:
        # layer format: [from, repeats, module, args]
        if layer[2] == 'ADown':
            print(f"Replacing ADown layer: {layer}")
            # args for ADown is [out_channels], args for Conv is [out_channels, k=3, s=2]
            out_channels = layer[3][0]
            # According to paper: Conv [c2, 3, 2]
            new_layer = [layer[0], layer[1], 'Conv', [out_channels, 3, 2]]
            new_layers.append(new_layer)
        else:
            new_layers.append(layer)
    return new_layers

# 3. Apply replacement to backbone and head
print("Modifying Backbone...")
yolo_yaml['backbone'] = replace_adown(yolo_yaml['backbone'])
print("Modifying Head...")
yolo_yaml['head'] = replace_adown(yolo_yaml['head'])

# 4. Save proposed config
proposed_yaml = 'models/detect/yolov9-proposed.yaml'
with open(proposed_yaml, 'w') as f:
    yaml.dump(yolo_yaml, f)
print(f"Proposed architecture saved to {proposed_yaml}")

# 5. Train using the PROPOSED architecture
!python train.py \
    --workers 4 \
    --device 0 \
    --batch 8 \
    --data data/pothole.yaml \
    --img 640 \
    --cfg models/detect/yolov9-proposed.yaml \
    --weights './yolov9-c.pt' \
    --name pothole_yolov9_proposed_run \
    --hyp hyp.scratch-high.yaml \
    --min-items 0 \
    --epochs 50

## 6. Evaluation & Inference
Evaluate the model on the validation set.

In [ ]:
!python val.py \
    --data data/pothole.yaml \
    --img 640 \
    --batch 32 \
    --conf 0.001 \
    --iou 0.7 \
    --device 0 \
    --weights runs/train/pothole_yolov9_proposed_run/weights/best.pt \
    --save-json \
    --name pothole_val

### Display Results

In [ ]:
from IPython.display import Image
import os

confusion_matrix_path = 'runs/train/pothole_yolov9_proposed_run/confusion_matrix.png'
if os.path.exists(confusion_matrix_path):
    display(Image(filename=confusion_matrix_path, width=600))
else:
    print("Confusion matrix not found. Training might have failed or not started.")

In [ ]:
# Display sample predictions
pred_path = 'runs/train/pothole_yolov9_proposed_run/val_batch0_pred.jpg'
if os.path.exists(pred_path):
    display(Image(filename=pred_path, width=800))
else:
    print("Prediction sample not found.")

In [ ]:
# Run inference on a test image
import glob
test_images = glob.glob('../test/images/*.jpg')
if test_images:
    test_image = test_images[0]
    !python detect.py \
        --source "{test_image}" \
        --img 640 \
        --device 0 \
        --weights runs/train/pothole_yolov9_proposed_run/weights/best.pt \
        --name pothole_detect
else:
    print("No test images found to run inference on.")